<a href="https://colab.research.google.com/github/elizamarghidanu/ElizaPortfolio/blob/master/ThesisCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
### 1. Mount Google Drive ###

from google.colab import drive

drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [2]:
ROOT_DIR = '/content/gdrive/My Drive/Colab_Notebooks'

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install ultralytics

In [ ]:
import os

from ultralytics import YOLO


# Load a model
model = YOLO("yolov8n.pt")  # load pre trained model

# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "google_colab_config.yaml"), epochs=20, imgsz=640, batch=16)  # train the model





In [ ]:

import cv2
import os
import numpy as np


model = YOLO('yolov8n.pt')

#define a directory containing images to test or an image file directly
input_dir = '/content/gdrive/My Drive/Colab_Notebooks/InputImages/train'
os.makedirs(input_dir, exist_ok=True)  # Make sure the directory exists

output_dir = '/content/gdrive/My Drive/Colab_Notebooks/PredictedImages'
os.makedirs(output_dir, exist_ok=True)

output_dir_grouped = '/content/gdrive/My Drive/Colab_Notebooks/PredictedGroups'

group_text_dir = '/content/gdrive/My Drive/Colab_Notebooks/AnnotatedGroups'

#loop through images and perform inference
image_files = [file for file in os.listdir(input_dir) if file.lower().endswith(('.jpg', '.jpeg', '.png'))]
for image_file in image_files:

    image_path = os.path.join(input_dir, image_file)
    image = cv2.imread(image_path)

    #perform inference using the YOLOv8n model
    results_list = model.predict(source=image_path)

    all_boxes = []
    names = results_list[0].names
    print(image_file)
    # Iterate through each result to draw bounding boxes for "person" class
    for results in results_list:
        for box in results.boxes:
            if results.names[int(box.cls)] == 'person':  # Check for "person" class
                x1, y1, x2, y2 = map(int, box.xyxy[0].cpu().numpy())
                confidence = box.conf
                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                #cv2.putText(image, f'Person: {confidence:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    for result in results_list:
        for box in result.boxes:
            if result.names[int(box.cls)] == 'person':
                coords = np.array(box.xyxy.cpu(), dtype=int).squeeze()
                all_boxes.append(coords)

    output_path = os.path.join(output_dir, image_file)
    cv2.imwrite(output_path, image)
    grouped_boxes = group_boxes(all_boxes)
    label = 'person'
    for group_idx, group in enumerate(grouped_boxes):
        print(group_idx)
        color = tuple(np.random.randint(0, 255, size=3).tolist())  # Random color for each group
        for box in group:
            left, top, right, bottom = box
            cv2.rectangle(image, (left, top), (right, bottom), color, 2)
            print(box)
            #label_idx = int(results2[0].pred[0][group_idx, 5])  # Adjust based on your results structure
            #label = names[label_idx]
            #confidence = float(results2[0].pred[0][group_idx, 4])
            cv2.putText(image, f'{label}: group {group_idx}', (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 2)


    output_path = os.path.join(output_dir_grouped, image_file)
    cv2.imwrite(output_path, image)

    group_text_path = os.path.join(group_text_dir, f'{os.path.splitext(image_file)[0]}_groups.txt')
    with open(group_text_path, 'w') as f:
        for group_idx, group in enumerate(grouped_boxes):
            f.write(f'Group {group_idx}:\n')
            for box in group:
                left, top, right, bottom = box
                f.write(f'\t{left}, {top}, {right}, {bottom}\n')


    # Save the annotated image back to Google Drive

print(f"Processed {len(image_files)} images and saved them to {output_dir}")

In [8]:
import os

def read_predicted_groups_from_txt(txt_dir):
    """
    Reads predicted groups from .txt files in the specified directory.

    Args:
        txt_dir (str): Path to the directory containing .txt files.

    Returns:
        dict: A dictionary where each key is an image file name and the value is a dict of group IDs and bounding boxes.
    """
    predictions = {}

    # Iterate through each .txt file in the directory
    for txt_file in os.listdir(txt_dir):
        if txt_file.lower().endswith('.txt'):
            image_name = os.path.splitext(txt_file)[0]
            predictions[image_name] = {}

            # Read the contents of the .txt file
            with open(os.path.join(txt_dir, txt_file), 'r') as file:
                current_group = None

                for line in file:
                    line = line.strip()
                    if not line:
                        continue

                    if line.startswith('Group'):
                        # Extract group ID
                        current_group = line.split()[1][:-1]
                        predictions[image_name][current_group] = []
                    else:
                        # Parse bounding box coordinates
                        try:
                            coords = [int(value.strip()) for value in line.split(',')]
                            if len(coords) == 4 and current_group is not None:
                                predictions[image_name][current_group].append(tuple(coords))
                        except ValueError:
                            print(f"Skipping malformed coordinates line in {txt_file}: {line}")

    return predictions

# Example usage
predicted_txt_dir = '/content/gdrive/My Drive/Colab_Notebooks/AnnotatedGroups'  # Adjust to point to the directory containing your .txt files
predicted_groups = read_predicted_groups_from_txt(predicted_txt_dir)

# Display results for verification
for image, groups in predicted_groups.items():
    print(f'Image: {image}')
    for group, boxes in groups.items():
        print(f'  Group {group}: {boxes}')


Image: 153_groups
  Group 0: [(146, 498, 214, 608)]
Image: 154_groups
  Group 0: [(238, 89, 443, 425)]
Image: 155_groups
  Group 0: [(179, 166, 236, 317), (116, 139, 199, 372), (219, 197, 250, 275)]
Image: 156_groups
  Group 0: [(69, 29, 439, 420), (410, 1, 597, 312)]
Image: 157_groups
Image: 158_groups
  Group 0: [(428, 216, 454, 241)]
  Group 1: [(245, 211, 270, 237)]
Image: 159_groups
  Group 0: [(223, 32, 498, 526)]
  Group 1: [(491, 97, 622, 522), (566, 153, 639, 524)]
  Group 2: [(0, 174, 49, 262), (14, 157, 106, 389), (0, 223, 103, 522)]
  Group 3: [(184, 151, 260, 264)]
Image: 160_groups
  Group 0: [(94, 129, 543, 633)]
Image: 161_groups
  Group 0: [(145, 274, 278, 629)]
Image: 162_groups
  Group 0: [(203, 36, 566, 480)]
Image: 163_groups
  Group 0: [(123, 142, 185, 306)]
Image: 164_groups
  Group 0: [(182, 58, 226, 198)]
  Group 1: [(73, 60, 117, 190), (111, 41, 129, 76)]
  Group 2: [(239, 74, 371, 317), (312, 74, 377, 243), (232, 41, 256, 90), (287, 42, 304, 77)]
  Group 3: [

In [38]:
import shutil

source_path = '/content/runs'
destination_path = '/content/gdrive/My Drive/Colab_Notebooks/runs7'
drive_dir_grouped = '/content/gdrive/My Drive/Colab_Notebooks/GroupedImages'

os.makedirs(drive_dir_grouped, exist_ok=True)

# Make sure the destination directory exists, if not create it
shutil.copytree(source_path, destination_path)



'/content/gdrive/My Drive/Colab_Notebooks/runs7'

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

image = cv2.imread('/content/gdrive/My Drive/Colab_Notebooks/imgtest.jpeg')

results2 = model.predict(image)

for result in results2:
            for box in result.boxes:
                left, top, right, bottom = np.array(box.xyxy.cpu(), dtype=int).squeeze()
                width = right - left
                height = bottom - top
                center = (left + int((right-left)/2), top + int((bottom-top)/2))
                label = results2[0].names[int(box.cls)]
                confidence = float(box.conf.cpu())
                cv2.rectangle(image, (left, top),(right, bottom), (255, 0, 0), 2)

            cv2_imshow(image)
            cv2.waitKey(0)


In [12]:
import json
import os

def load_ground_truth(json_dir):
    ground_truth_data = {}
    for json_file in os.listdir(json_dir):
        if json_file.endswith('.json'):
            file_path = os.path.join(json_dir, json_file)
            with open(file_path, 'r') as file:
                data = json.load(file)
                image_name = data["asset"]["name"]
                ground_truth_data[image_name] = {}

                for region in data["regions"]:
                    group_tag = region["tags"][0]  # Assuming one tag per region
                    bbox = region["boundingBox"]
                    bbox_coords = (
                        int(bbox["left"]),
                        int(bbox["top"]),
                        int(bbox["left"] + bbox["width"]),
                        int(bbox["top"] + bbox["height"])
                    )
                    if group_tag not in ground_truth_data[image_name]:
                        ground_truth_data[image_name][group_tag] = []
                    ground_truth_data[image_name][group_tag].append(bbox_coords)
    print(ground_truth_data)
    return ground_truth_data

json_dir = '/content/gdrive/My Drive/Colab_Notebooks/group_truth_labels'  # Path to your JSON files directory
ground_truths = load_ground_truth(json_dir)


{'153.jpeg': {'Group 0': [(133, 489, 222, 616)]}, '154.jpeg': {'Group 0': [(207, 68, 432, 457)]}, '155.jpeg': {'Group 0': [(65, 73, 159, 420), (119, 123, 195, 379), (177, 163, 235, 320), (216, 201, 248, 279), (237, 220, 258, 264)]}, '156.jpeg': {'Group 0': [(55, 32, 463, 425), (381, 0, 601, 333)]}, '158.jpeg': {'Group 0': [(422, 212, 460, 244)], 'Group 1': [(238, 210, 274, 237)]}, '159.jpeg': {'Group 0': [(165, 28, 499, 527)], 'Group 1': [(560, 182, 640, 525), (505, 185, 606, 527)], 'Group 2': [(0, 160, 62, 274), (11, 148, 131, 405), (0, 45, 44, 527)], 'Group 3': [(145, 139, 261, 274)]}, '160.jpeg': {'Group 0': [(81, 105, 558, 640)]}, '161.jpeg': {'Group 0': [(135, 268, 281, 639)]}, '162.jpeg': {'Group 0': [(194, 31, 577, 480)]}, '163.jpeg': {'Group 0': [(108, 136, 193, 319)]}, '164.jpeg': {'Group 2': [(302, 70, 369, 257), (234, 71, 400, 321), (229, 40, 260, 95), (284, 41, 310, 84)], 'Group 0': [(180, 59, 224, 201)], 'Group 1': [(70, 58, 118, 193), (106, 37, 130, 77)]}, '165.jpeg': {'G

In [5]:
def boxes_intersect(box1, box2):
    # Unpack coordinates
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2

    # Check for intersection
    intersection = not (x2 < x3 or x1 > x4 or y2 < y3 or y1 > y4)
    if (intersection):
        distanc = abs(y1 - y3)
        #print(distance)
    return ((not (x2 < x3 or x1 > x4 or y2 < y3 or y1 > y4)) and (distanc < 60))


In [6]:

def group_boxes(boxes):
    groups = []
    used = set()

    for i in range(len(boxes)):
        if i in used:
            continue
        current_group = [boxes[i]]
        stack = [boxes[i]]
        used.add(i)

        while stack:
            current = stack.pop()
            for j in range(len(boxes)):
                if j not in used and boxes_intersect(current, boxes[j]):
                    used.add(j)
                    stack.append(boxes[j])
                    current_group.append(boxes[j])

        groups.append(current_group)
    return groups


In [19]:
def compute_iou(box1, box2):
    x1, y1, x2, y2 = box1
    xx1, yy1, xx2, yy2 = box2
    xi1, yi1 = max(x1, xx1), max(y1, yy1)
    xi2, yi2 = min(x2, xx2), min(y2, yy2)
    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (xx2 - xx1) * (yy2 - yy1)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

def evaluate_predictions(predicted_groups, ground_truths):
    # Normalize keys by removing any suffix like '_groups' and ensuring file extension consistency
    normalized_predicted_groups = {key.split('_')[0] + '.jpeg': value for key, value in predicted_groups.items()}

    # Initialize dictionaries to store precision, recall, and F1 for each image
    precision_scores = {}
    recall_scores = {}
    f1_scores = {}

    results = {}
    for image_name, gt_groups in ground_truths.items():
        pred_groups = normalized_predicted_groups.get(image_name, {})
        results[image_name] = {'tp': 0, 'fp': 0, 'fn': 0}
        for gt_group_id, gt_boxes in gt_groups.items():
            matched = False
            for pred_group_id, pred_boxes in pred_groups.items():
                for gt_box in gt_boxes:
                    for pred_box in pred_boxes:
                        iou = compute_iou(gt_box, pred_box)
                        if iou > 0.5:
                            results[image_name]['tp'] += 1
                            matched = True
                            break
                    if matched:
                        break
                if matched:
                    break
            if not matched:
                results[image_name]['fn'] += 1
        results[image_name]['fp'] = sum(len(v) for v in pred_groups.values()) - results[image_name]['tp']

        # Calculate precision, recall, and F1 score for each image
        tp = results[image_name]['tp']
        fp = results[image_name]['fp']
        fn = results[image_name]['fn']
        precision = tp / (tp + fp) if tp + fp > 0 else 0
        recall = tp / (tp + fn) if tp + fn > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        # Store the scores
        precision_scores[image_name] = precision
        recall_scores[image_name] = recall
        f1_scores[image_name] = f1_score

    # Calculate average scores
    average_precision = sum(precision_scores.values()) / len(precision_scores)
    average_recall = sum(recall_scores.values()) / len(recall_scores)
    average_f1_score = sum(f1_scores.values()) / len(f1_scores)

    return average_precision, average_recall, average_f1_score

# Call the function and print the average scores
average_precision, average_recall, average_f1_score = evaluate_predictions(predicted_groups, ground_truths)
print(f"Average Precision: {average_precision:.2f}")
print(f"Average Recall: {average_recall:.2f}")
print(f"Average F1 Score: {average_f1_score:.2f}")


Average Precision: 0.76
Average Recall: 0.98
Average F1 Score: 0.83


In [18]:
def compute_iou(box1, box2):
    x1, y1, x2, y2 = box1
    xx1, yy1, xx2, yy2 = box2
    xi1, yi1 = max(x1, xx1), max(y1, yy1)
    xi2, yi2 = min(x2, xx2), min(y2, yy2)
    inter_area = max(xi2 - xi1, 0) * max(yi2 - yi1, 0)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (xx2 - xx1) * (yy2 - yy1)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

def evaluate_predictions(predicted_groups, ground_truths):
    # Normalize keys by removing any suffix like '_groups' and ensuring file extension consistency
    normalized_predicted_groups = {key.split('_')[0] + '.jpeg': value for key, value in predicted_groups.items()}

    results = {}
    for image_name, gt_groups in ground_truths.items():
        pred_groups = normalized_predicted_groups.get(image_name, {})
        results[image_name] = {'tp': 0, 'fp': 0, 'fn': 0}
        for gt_group_id, gt_boxes in gt_groups.items():
            matched = False
            for pred_group_id, pred_boxes in pred_groups.items():
                for gt_box in gt_boxes:
                    for pred_box in pred_boxes:
                        iou = compute_iou(gt_box, pred_box)
                        print(f"Comparing GT box {gt_box} with Pred box {pred_box}, IoU: {iou}")
                        if iou > 0.5:
                            results[image_name]['tp'] += 1
                            matched = True
                            break
                    if matched:
                        break
                if matched:
                    break
            if not matched:
                results[image_name]['fn'] += 1
        results[image_name]['fp'] = sum(len(v) for v in pred_groups.values()) - results[image_name]['tp']
    return results

evaluation_results = evaluate_predictions(predicted_groups, ground_truths)

for image, metrics in evaluation_results.items():
    tp = metrics['tp']
    fp = metrics['fp']
    fn = metrics['fn']
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    print(f"Results for {image}:")
    print(f"  Precision: {precision:.2f}")
    print(f"  Recall: {recall:.2f}")
    print(f"  F1 Score: {f1_score:.2f}")


Comparing GT box (133, 489, 222, 616) with Pred box (146, 498, 214, 608), IoU: 0.6617712111828719
Comparing GT box (207, 68, 432, 457) with Pred box (238, 89, 443, 425), IoU: 0.7145723024303614
Comparing GT box (65, 73, 159, 420) with Pred box (179, 166, 236, 317), IoU: 0.0
Comparing GT box (65, 73, 159, 420) with Pred box (116, 139, 199, 372), IoU: 0.2389002813677333
Comparing GT box (65, 73, 159, 420) with Pred box (219, 197, 250, 275), IoU: 0.0
Comparing GT box (119, 123, 195, 379) with Pred box (179, 166, 236, 317), IoU: 0.09420205092213514
Comparing GT box (119, 123, 195, 379) with Pred box (116, 139, 199, 372), IoU: 0.83975909328022
Comparing GT box (55, 32, 463, 425) with Pred box (69, 29, 439, 420), IoU: 0.8891696706182566
Comparing GT box (422, 212, 460, 244) with Pred box (428, 216, 454, 241), IoU: 0.5345394736842105
Comparing GT box (238, 210, 274, 237) with Pred box (428, 216, 454, 241), IoU: 0.0
Comparing GT box (238, 210, 274, 237) with Pred box (245, 211, 270, 237), IoU: